# 一个仅基于Numpy的MNIST手写数字识别

为了深入理解dnn中各种模块的作用与原理,在使用Pytorch或者TensorFlow等框架构建了简单的神经网络后,可以尝试仅使用Numpy辅助矩阵运算来构建一个与使用模块构建的网络功能完全一致的简单神经网络.这样可以更加深入理解深度学习的原理

## 函数定义
    1.将所需用到的所有的激活函数定义在(/Activationn_Function.ipynb)中
    2.将梯度计算方法定义在(/Gradientn_Calculating.ipynb)中
    3.将损失函数定义在(/Lossn_Function.ipynb)中
    4.将神经网络主题定义在(/Minstn_Reconition(basee on Numpy only).ipynb)中

## 外部库导入

In [1]:
import numpy as np
from tf.keras.datasets import mnist #仅为了加载数据集

## 本地模块导入	

In [4]:
import Activation_Function as af
import Gradient_Calculating as gc
import Loss_Function as lf

In [5]:
class TwoLayerNet:

    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
        # 初始化权重
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)

    def predict(self, x):
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']

        a1 = np.dot(x, W1) + b1
        #z1 = af.sigmoid(a1)
        z1 = af.relu(a1)
        a2 = np.dot(z1, W2) + b2
        y = af.softmax(a2)

        return y

    # x:输入数据, t:监督数据
    def loss(self, x, t):
        y = self.predict(x)

        return cross_entropy_error(y, t)

    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        t = np.argmax(t, axis=1)

        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy

    # x:输入数据, t:监督数据
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)

        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])

        return grads

    def gradient(self, x, t):
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']
        grads = {}

        batch_num = x.shape[0]

        # forward
        a1 = np.dot(x, W1) + b1
        #z1 = af.sigmoid(a1)
        z1 = af.relu(a1)
        a2 = np.dot(z1, W2) + b2
        y = af.softmax(a2)

        # backward
        dy = (y - t) / batch_num
        grads['W2'] = np.dot(z1.T, dy)
        grads['b2'] = np.sum(dy, axis=0)

        da1 = np.dot(dy, W2.T)
        #dz1 = af.sigmoid_grad(a1) * da1
        dz1 = af.relu_grad(a1) * da1
        grads['W1'] = np.dot(x.T, dz1)
        grads['b1'] = np.sum(dz1, axis=0)

        return grads



## 转化独热编码函数

In [6]:
def _change_one_hot_label(X):
    T = np.zeros((X.size, 10))
    for idx, row in enumerate(T):
        row[X[idx]] = 1

    return T

## 载入数据

In [ ]:
#训练集
train_dataset=datasets.MNIST(root='./',#等号右侧为数据集的路径,./表示当前路径
                             train=True,#
                             transform=transforms.ToTensor(),#将MNIST数据集中数据转化为tensor数据
                             download=True#若路径不存在数据集,则下载数据
)
#测试集
test_dataset=datasets.MNIST(root='./',
                             train=False,
                             transform=transforms.ToTensor(),
                             download=True
)